# Assignment 4 (solutions): Attention

### Due Date: Nov 22 (both sections)

### Grade (100 pts, 10%)

#### Your Name:

#### Your EID:

*Note: This assignment covers material from the recording, notes, demo, and suggested readings from Lectures 5,8,9*

---

### Problem introduction

In this assignment we will explore the concept of attention using Word2Vec features. Recall that in lecture-05 we used the Word2Vec algorithm to compute feature representations of english words. For every center-word - context-word pair, ${\textbf{x}_w, \textbf{x}_c}$, in the training set, Word2Vec tries to maximize the inner product between their feature embeddings, $\textbf{u}_w$ and $\textbf{v}_c$, by first computing a vector of logits as $\textbf{z} = \textbf{u}_w \cdot \textbf{V}^T \in \mathbb{R}^N$, and then maximizing $p(\textbf{x}_c | \textbf{x}_w) = \sigma_{softmax}(\textbf{z})$ to learn $\textbf{U}$ and $\textbf{V}$ via MLE. 

Let's assume we are given a sequence of words:

$$X = \{ \textbf{x}^{(1)}, \textbf{x}^{(2)}, \textbf{x}^{(3)}, \textbf{x}^{(4)}, \dots, \textbf{x}^{(T)} \}$$

a corresponding sequence of center-word feature representations:

$$U = \{ \textbf{u}^{(1)}, \textbf{u}^{(2)}, \textbf{u}^{(3)}, \textbf{u}^{(4)}, \dots, \textbf{u}^{(T)} \}$$

and a corresponding context-word feature representations:

$$V = \{ \textbf{v}^{(1)}, \textbf{v}^{(2)}, \textbf{v}^{(3)}, \textbf{v}^{(4)}, \dots, \textbf{v}^{(T)} \}$$

Then, using the Word2Vec variable naming convention above, recall that in simple self attention (Lecture-09) we compute a set of *context vectors*:

$$C = \{ \textbf{c}^{(1)}, \textbf{c}^{(2)}, \textbf{c}^{(3)}, \textbf{c}^{(4)}, \dots, \textbf{c}^{(T)} \}$$ 

each of which is computed according to:

$$
\textbf{c}^{(t)} = \sum_{t'=1}^{T} \alpha_{t,t'} \; \textbf{u}^{(t)} \quad where \quad \alpha_{t,t'} = \frac{ e^{ \textbf{u}^{(t)} \cdot \textbf{v}^{(t')} } }{ \sum_{t''} e^{ \textbf{u}^{(t)} \cdot \textbf{v}^{(t'')} } } \\
$$

The attention weights make up the following (non-symmetric!) matrix:

$$
\textbf{A} = 
\left[ \begin{array}{ccc}
\alpha_{1,1} & \dots & \alpha_{1,T} \\
\vdots & \ddots & \vdots \\
\alpha_{T,1} & \dots & \alpha_{T,T}
\end{array}\right] \; \in [0,1]^{T \times T} \quad where \quad \sum_{t'} \alpha_{t,t'} = 1
$$

### Test sentence

In [1]:
sentence = "Thomas Jefferson was an American statesman and Founding Father who served as the third president of the United States"

### Load Word2Vec vocab, embedding matrices

In the `assignments/a4/` folder you will find three .tsv files: a vocabulary (`metadata.tsv`), center-word Word2Vec embeddings (`vectors_center.tsv`), and context-word Word2Vec embeddings (`vectors_context.tsv`). These are taking from the Lecture-05 Word2Vec demo. The code below loads the center and context word embeddings for the above test sentence into two numpy arrays: $U$, and $V$. 

In [2]:
import numpy as np

def load_matrix(fpath):
    matrix = []
    with open(fpath, 'r') as fd:
        tsv = fd.read()
    for line in tsv.split('\n'):
        row = []
        for value in line.split('\t'):
            row.append(float(value))
        matrix.append(row)
    return np.array(matrix)

def load_vocab(fpath) -> dict:
    with open(fpath, "r") as fd:
        tsv = fd.read()
    vocab = {}
    for line in tsv.split('\n'):
        vocab[line.strip()] = len(vocab)
    return vocab

vocab = load_vocab("metadata.tsv")
U_lookup = load_matrix("vectors_center.tsv").T
V_lookup = load_matrix("vectors_context.tsv").T

N = len(vocab)
D = len(U_lookup)

print(f"vocab size: {N}")
print(f"embed dim: {D}")

print(f"center embedding lookup matrix shape: {U_lookup.shape}")
print(f"context embedding lookup matrix shape: {V_lookup.shape}")

vocab size: 2321
embed dim: 15
center embedding lookup matrix shape: (15, 2321)
context embedding lookup matrix shape: (15, 2321)


In [3]:
word_list = sentence.lower().split(" ")
X = [vocab.get(word, N) for word in word_list]
print(f"Word indices: {X}")

Word indices: [2092, 1075, 2321, 2321, 79, 2321, 2321, 795, 2321, 2321, 2321, 2321, 2321, 2321, 1622, 2321, 2321, 2158, 1987]


In [4]:
# Assign OOV words low attention weights
u_oov = 0.1 * np.mean(U_lookup, axis=1, keepdims=True)
v_oov = 0.1 * np.mean(V_lookup, axis=1, keepdims=True) 
# Add the OOV embeddings to the lookup matrices
U_lookup = np.concatenate((U_lookup, u_oov), axis=1)
V_lookup = np.concatenate((V_lookup, v_oov), axis=1)
U_lookup.shape, V_lookup.shape

print(f"center word embedding lookup matrix w/oov embed added shape: {U_lookup.shape}")
print(f"context word embedding lookup matrix w/oov embed added shape: {U_lookup.shape}")

center word embedding lookup matrix w/oov embed added shape: (15, 2322)
context word embedding lookup matrix w/oov embed added shape: (15, 2322)


In [5]:
# Center-word embedding sequence of X
U = U_lookup[:, X]
# Context-word embedding sequence of X
V = V_lookup[:, X]

print(f"sequence of center-word embeddings shape: {U.shape}")
print(f"sequence of context-word embeddings shape: {U.shape}")

sequence of center-word embeddings shape: (15, 19)
sequence of context-word embeddings shape: (15, 19)


# Questions

### (40 pts) Q1: Attention weights

1. Compute the attention weight matrix $\textbf{A}$ for this test sentence using the definitions provided in the intro (and in Lecture-09 as needed).

*Hint: You can use the batched `softmax()` function provided below to compute A.*

In [6]:
def softmax(Z) -> np.ndarray:
    """
    Computes a softmax over each row of Z
    """
    Z_exp = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    partition = np.sum(Z_exp, axis=1, keepdims=True)
    return Z_exp / partition

In [7]:
# Your code goes here
A = []
for i in range(U.shape[0]):
    row = []
    u = U[i]
    denominator = sum([np.exp(np.dot(u, v)) for v in V])
    for i in range(V.shape[0]):
        v = V[i]
        numerator = np.exp(np.dot(u, v))
        alpha = numerator / denominator
        row.append(alpha)
    row = np.array(row)
    A.append(row)
A = np.array(A)
A

array([[1.38194540e-01, 2.03212257e-01, 3.02180741e-01, 1.85227602e-02,
        7.88875829e-02, 5.05436377e-03, 5.39829404e-02, 4.61455844e-02,
        1.66098147e-02, 6.46488339e-03, 6.18762017e-02, 1.67377658e-02,
        4.31444090e-03, 2.94915922e-02, 1.83245312e-02],
       [3.36422878e-02, 4.00281650e-01, 5.32444420e-02, 6.42213438e-03,
        2.15077226e-01, 1.38693578e-02, 9.22402640e-02, 3.43752852e-02,
        3.28024493e-02, 1.56717667e-03, 7.62786516e-02, 1.60550691e-02,
        2.69083957e-03, 1.53324575e-02, 6.12070950e-03],
       [5.44450146e-02, 7.84255340e-02, 5.09397467e-01, 9.87708773e-03,
        1.78759849e-01, 7.78388085e-03, 3.23163023e-02, 4.57153269e-02,
        4.21990112e-03, 2.98786600e-03, 4.66210300e-02, 7.75445102e-03,
        1.60603487e-03, 1.08856662e-02, 9.20458825e-03],
       [5.06306216e-02, 6.98592499e-02, 5.83901884e-02, 7.14002572e-02,
        1.15119950e-01, 7.15920391e-02, 6.69789141e-02, 9.44154984e-02,
        5.44823143e-02, 4.57178135e-0

### (40 pts) Q2: Attention block

1. Now compute the attention block $C$ using the attention weights that you've computed

2. Compute the L2 norm of each of the resultant context vectors. Do the same for the center-word embeddings in the preceeding layer. How do the magnitudes compare? Is there a logical explanation for this?

*Hint: You need to compute $\textbf{c}^{(t)} \; \forall t \in \{1, \dots, T\}$.*

In [8]:
# Your code goes here
C = []
for i in range(U.shape[0]):
    C.append(np.sum(A[i]) * U[i])
C = np.array(C)
C

array([[ 1.36663999,  0.37958566,  0.04843033,  0.04843033,  0.64411976,
         0.04843033,  0.04843033, -0.55828937,  0.04843033,  0.04843033,
         0.04843033,  0.04843033,  0.04843033,  0.04843033, -0.11014929,
         0.04843033,  0.04843033,  0.65544686,  0.623705  ],
       [ 1.40334786,  0.69007441,  0.04996142,  0.04996142,  0.75487875,
         0.04996142,  0.04996142,  0.6100362 ,  0.04996142,  0.04996142,
         0.04996142,  0.04996142,  0.04996142,  0.04996142,  1.06472199,
         0.04996142,  0.04996142,  0.21640897,  0.23620917],
       [ 0.55068026,  0.84642704,  0.05148176,  0.05148176,  0.58390371,
         0.05148176,  0.05148176,  0.2392508 ,  0.05148176,  0.05148176,
         0.05148176,  0.05148176,  0.05148176,  0.05148176, -0.44037865,
         0.05148176,  0.05148176,  1.04444346,  0.93345815],
       [ 0.11657596, -0.08050063,  0.04746895,  0.04746895, -0.16628096,
         0.04746895,  0.04746895,  0.59962645,  0.04746895,  0.04746895,
         0.047

In [9]:
C_L2 = []
for c in C:
    scaling_factor = sum(c * c)
    c /= scaling_factor
    C_L2.append(c)
C_L2 = np.array(C_L2)
C_L2

array([[ 0.37991087,  0.10552063,  0.0134631 ,  0.0134631 ,  0.1790582 ,
         0.0134631 ,  0.0134631 , -0.1551983 ,  0.0134631 ,  0.0134631 ,
         0.0134631 ,  0.0134631 ,  0.0134631 ,  0.0134631 , -0.03062029,
         0.0134631 ,  0.0134631 ,  0.18220701,  0.17338312],
       [ 0.30154957,  0.1482823 ,  0.01073564,  0.01073564,  0.16220737,
         0.01073564,  0.01073564,  0.13108379,  0.01073564,  0.01073564,
         0.01073564,  0.01073564,  0.01073564,  0.01073564,  0.22878608,
         0.01073564,  0.01073564,  0.04650168,  0.05075632],
       [ 0.15272001,  0.23473939,  0.01427742,  0.01427742,  0.16193386,
         0.01427742,  0.01427742,  0.06635136,  0.01427742,  0.01427742,
         0.01427742,  0.01427742,  0.01427742,  0.01427742, -0.12213009,
         0.01427742,  0.01427742,  0.28965523,  0.2588757 ],
       [ 0.26083505, -0.18011763,  0.10621028,  0.10621028, -0.37204841,
         0.10621028,  0.10621028,  1.34164531,  0.10621028,  0.10621028,
         0.106

In [10]:
U_L2 = []
for u in U:
    scaling_factor = sum(u * u)
    u /= scaling_factor
    U_L2.append(u)
U_L2 = np.array(U_L2)
U_L2

array([[ 0.37991087,  0.10552063,  0.0134631 ,  0.0134631 ,  0.1790582 ,
         0.0134631 ,  0.0134631 , -0.1551983 ,  0.0134631 ,  0.0134631 ,
         0.0134631 ,  0.0134631 ,  0.0134631 ,  0.0134631 , -0.03062029,
         0.0134631 ,  0.0134631 ,  0.18220701,  0.17338312],
       [ 0.30154957,  0.1482823 ,  0.01073564,  0.01073564,  0.16220737,
         0.01073564,  0.01073564,  0.13108379,  0.01073564,  0.01073564,
         0.01073564,  0.01073564,  0.01073564,  0.01073564,  0.22878608,
         0.01073564,  0.01073564,  0.04650168,  0.05075632],
       [ 0.15272001,  0.23473939,  0.01427742,  0.01427742,  0.16193386,
         0.01427742,  0.01427742,  0.06635136,  0.01427742,  0.01427742,
         0.01427742,  0.01427742,  0.01427742,  0.01427742, -0.12213009,
         0.01427742,  0.01427742,  0.28965523,  0.2588757 ],
       [ 0.26083505, -0.18011763,  0.10621028,  0.10621028, -0.37204841,
         0.10621028,  0.10621028,  1.34164531,  0.10621028,  0.10621028,
         0.106

In [11]:
V_L2 = []
for v in V:
    scaling_factor = sum(v * v)
    v /= scaling_factor
    V_L2.append(v)
V_L2 = np.array(V_L2)
V_L2

array([[ 0.12782813,  0.04808949,  0.00528336,  0.00528336,  0.15044354,
         0.00528336,  0.00528336, -0.00136731,  0.00528336,  0.00528336,
         0.00528336,  0.00528336,  0.00528336,  0.00528336,  0.0531058 ,
         0.00528336,  0.00528336,  0.17093938,  0.1769661 ],
       [ 0.13001926,  0.07781141,  0.00371445,  0.00371445,  0.1303005 ,
         0.00371445,  0.00371445,  0.05036609,  0.00371445,  0.00371445,
         0.00371445,  0.00371445,  0.00371445,  0.00371445,  0.09410843,
         0.00371445,  0.00371445,  0.1050567 ,  0.11141972],
       [ 0.04834294,  0.0619845 ,  0.00297956,  0.00297956,  0.11760508,
         0.00297956,  0.00297956,  0.02034528,  0.00297956,  0.00297956,
         0.00297956,  0.00297956,  0.00297956,  0.00297956,  0.00955488,
         0.00297956,  0.00297956,  0.13808807,  0.13780891],
       [ 0.12503901, -0.07570418,  0.00874934,  0.00874934,  0.21590144,
         0.00874934,  0.00874934,  0.10268653,  0.00874934,  0.00874934,
         0.008

In [ ]:
# From a cursory glance (using the plotting function below), it seems like highs and lows appear in relatively the same places.
# They both seem to be the same order of magnitude, but U has higher peaks than V, which is more spread out.
# This makes some sense logically, because Center-word embeddings would peak up at fewer, more important words, while 
# Context-word embeddings would spread out much more, especially given just a single sample sentence, where each word is 
# relevant to the whole contextually.

### (20 pts) Q3: Visualize the attention weights

Now that we've computed a simple self attention transformation of our Word2Vec sequence representation, it's time to visually examine what this attention mechanism affords us. First recognize that all values in $\textbf{A}$ are in the range $[0,1]$, and that each row is normalized. Therefore, we really just need to generate a heatmap of this matrix, keeping in mind that the matrix indices correspond directly to the positions in the sequence.

1. Generate the heatmap in the cell provided below

2. Briefly describe/explain what you see in the heatmap.

3. Why is our attention matrix not symmetric?

*Hint: You can generate a heatmap of the attention weights by calling `plot_attention_weights(A)`.*

In [13]:
import plotly.express as px

def plot_attention_weights(att_matrix):
    fig = px.imshow(att_matrix)#, x=word_list, y=word_list, width=20, height=20)
    fig.update_layout(width=800, height=800)
    print(sentence)
    fig.show()

In [15]:
# Your code goes here
plot_attention_weights(A)
# There are some words that are very important in the matrix, and they appear to be important relative to each of the 
# other words in the sentence. 
# The heatmap/matrix isn't symmetric because the Attention matrix represents each word's importance with regards to the others.
# "President" provides more information to "Third" than "Third" does to "President". This asymmetric relationship is 
# reflected in the matrix/heatmap.

Thomas Jefferson was an American statesman and Founding Father who served as the third president of the United States


In [17]:
plot_attention_weights(U_L2)
plot_attention_weights(V_L2)
# Visualizing U and V for question 2

Thomas Jefferson was an American statesman and Founding Father who served as the third president of the United States


Thomas Jefferson was an American statesman and Founding Father who served as the third president of the United States
